In [1]:
import torchaudio as ta
import torchaudio.transforms as T
import torch
import numpy as np
import yaml
# import mir_eval
import gc

import warnings
# from train import RTBWETrain
# from datamodule import *
from utils import *

from tqdm import tqdm
import wandb
from pesq import pesq
from pystoi import stoi
import random
from torch.utils.data import Subset
import soundfile as sf
from datetime import datetime
import sys
import torch.nn.functional as F
import argparse

from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler
# from SEANet_v2 import SEANet_ver2
from MelGAN import Discriminator_MelGAN
from MBSTFTD import MultiBandSTFTDiscriminator

from dataset import CustomDataset

# from models.SEANet_TFiLM import SEANet_TFiLM
from models.SEANet import SEANet
from models.SEANet_TFiLM_nok_modified import SEANet_TFiLM 

def load_model(model, checkpoint_path):
    # model = model.to(DEVICE)
    ckpt = torch.load(checkpoint_path)
    model.load_state_dict(ckpt['generator_state_dict'])
    print(f"Model loaded from {checkpoint_path}")
    return model

def count_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    params_m = total_params / 1000000
    print(f"Total {params_m:.2f}M Parameters !")
    return total_params

/home/woongjib/anaconda3/envs/env2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-20 21:05:47.609975: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-20 21:05:48.296786: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [15]:
from torchinfo import summary

model = SEANet_TFiLM(visualize=False)

wb_p = [
            "/home/woongjib/Projects/Dataset/FSD50K_WB_SEGMENT/", 
            "/home/woongjib/Projects/Dataset/MUSDB_WB_SEGMENT/", 
            ]
nb_p = [
            "/home/woongjib/Projects/Dataset/FSD50K_LPF/", 
            "/home/woongjib/Projects/Dataset/MUSDB_LPF/", 
            ]
dataset = CustomDataset(path_dir_nb=nb_p, path_dir_wb=wb_p, seg_len=0.9, mode="train", high_index=31)

for wb,nb,spec,_,_ in dataset:
    print((wb.shape))
    print(spec.shape)    
    break

y = model(nb,spec)
count_model_parameters(model)

model = SEANet_TFiLM(min_dim=8, visualize=False, in_channels=64)
# out = model(nb, spec)
print(summary(model, input_data=[nb,spec]))

print('\n')
model = SEANet_TFiLM(min_dim=16, visualize=False, in_channels=32)
print(summary(model, input_data=[nb,spec]))



Index:0 with 283048 samples
Index:1 with 144773 samples
LR 427821 and HR 427821 file numbers loaded!
train: 427821 files loaded
torch.Size([1, 43200])
torch.Size([1, 832, 21])
Total 2.23M Parameters !
Layer (type:depth-idx)                        Output Shape              Param #
SEANet_TFiLM                                  [1, 1, 43200]             --
├─ResNet: 1-1                                 [1, 512, 26, 21]          --
│    └─CausalConv2d: 2-1                      [1, 64, 416, 21]          3,200
│    └─ReLU: 2-2                              [1, 64, 416, 21]          --
│    └─MaxPool2d: 2-3                         [1, 64, 208, 21]          --
│    └─Sequential: 2-4                        [1, 64, 208, 21]          --
│    │    └─BasicBlock: 3-1                   [1, 64, 208, 21]          73,856
│    │    └─BasicBlock: 3-2                   [1, 64, 208, 21]          73,856
│    └─Sequential: 2-5                        [1, 128, 104, 21]         --
│    │    └─BasicBlock: 3-3      

In [12]:
enc = model.encoder
count_model_parameters(enc)

Total 1.00M Parameters !


996800